In [7]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from skimage.transform import resize

In [8]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    proc_img = ((img/255)-img_mean)/img_std
    proc_img = resize(proc_img, img_size)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    with open(model_path, 'r') as json_file:
         model = keras.models.model_from_json(json_file.read())  
    model.load_weights(weight_path)
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    pred = model.predict(img)
    if pred > thresh:
        prediction = 'Pneumonia'
    else:
        prediction = 'Non-pneumonia'
    
    return prediction 

In [11]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json" #path to saved model
weight_path = "{}_my_model.best.hdf5".format('xray_class') #path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0 # loads the mean image value they used during training preprocessing
img_std = 1 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
my_model.thresh = 0.5
print("thresh:", my_model.thresh)
thresh = my_model.thresh #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    actual = pydicom.dcmread(i).StudyDescription    
    print(f'''Model predicted {pred} with thresh = {thresh:.2}, while the actual case is {actual}''')

thresh: 0.5
Load file test1.dcm ...
Model predicted Non-pneumonia with thresh = 0.5, while the actual case is No Finding
Load file test2.dcm ...
Model predicted Non-pneumonia with thresh = 0.5, while the actual case is Cardiomegaly
Load file test3.dcm ...
Model predicted Non-pneumonia with thresh = 0.5, while the actual case is Effusion
Load file test4.dcm ...
Model predicted Non-pneumonia with thresh = 0.5, while the actual case is No Finding
Load file test5.dcm ...
Model predicted Non-pneumonia with thresh = 0.5, while the actual case is No Finding
Load file test6.dcm ...
Model predicted Non-pneumonia with thresh = 0.5, while the actual case is No Finding
